<a href="https://colab.research.google.com/github/lucasfelipecdm/fiap-hackathon-vision-guard/blob/main/fiap_hackathon_visionguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install ultralytics
!pip install opencv-python
!pip install email_validator
!pip install yagmail

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
# Import libraries
import os
import cv2
from ultralytics import YOLO
import yagmail
from email_validator import validate_email, EmailNotValidError

# Mount Google Drive
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


In [3]:
# --- CONFIGURATION ---
data_path = userdata.get('DATA_PATH')
# Path to your dataset (replace with your actual path)
dataset_path = data_path + '/dataset3'  # Example: /content/drive/MyDrive/yolov8_dataset

# Path to your video (replace with your actual path)
video_path = data_path + '/test-video/video-test-1.mp4'  # Example: /content/drive/MyDrive/video.mp4
video_path2 = data_path + '/test-video/video-test-2.mp4'  # Example: /content/drive/MyDrive/video.mp4

# Email configuration (replace with your credentials)
SENDER_EMAIL = userdata.get('GMAIL_EMAIL')  # Your Gmail address
SENDER_PASSWORD = userdata.get('GMAIL_PASSWORD')  # Your Gmail password or App Password (recommended)
RECEIVER_EMAIL = "lucasfelipecdm@hotmail.com" # Recipient email address

# Classes to detect (adjust if needed)
CLASSES_TO_DETECT = ['knife']  # Match your dataset's classes

# --- NEW: Create directory in Google Drive ---
output_folder = data_path + '/results'  # Path to your desired folder
os.makedirs(output_folder, exist_ok=True)  # Create if it doesn't exist

In [4]:
# --- FUNCTIONS ---

def train_model(dataset_path):
    """Trains the YOLOv8 model."""
    model = YOLO('yolov8m.pt')  # You can change to a larger model like yolov8s.pt, yolov8m.pt, etc. for better accuracy if needed
    model.train(data=os.path.join(dataset_path, 'data.yaml'), epochs=10)  # Adjust epochs as needed
    return model

def detect_objects(model, video_path):
    """Detects objects in the video and sends an email if any are found."""
    cap = cv2.VideoCapture(video_path)
    object_detected = False
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                if class_name in CLASSES_TO_DETECT:
                    object_detected = True
                    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get box coordinates
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
                    cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)  # Add label
                    # --- NEW: Save the frame ---
                    frame_filename = f"frame_{frame_count}.jpg"  # Unique filename
                    frame_path = os.path.join(output_folder, frame_filename)
                    cv2.imwrite(frame_path, frame)  # Save the frame
                    frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'q' to quit
            break

    cap.release()
    cv2.destroyAllWindows()

    return object_detected


def send_email(object_detected):
    """Sends an email notification."""
    if object_detected:
         try:
            # Validate email addresses
            validated_sender = validate_email(SENDER_EMAIL)
            validated_receiver = validate_email(RECEIVER_EMAIL)

            # Use validated and normalized email addresses
            sender_email = validated_sender["email"]
            receiver_email = validated_receiver["email"]

            yag = yagmail.SMTP(user=sender_email, password=SENDER_PASSWORD)
            contents = ["Sharps objects detected in the video!"]  # Email content
            yag.send(to=receiver_email, subject="Sharps Object Detection Alert", contents=contents)
            print("Email sent successfully!")

         except EmailNotValidError as e:
            print(f"Invalid email address: {e}")
         except Exception as e:
             print(f"Error sending email: {e}")
    else:
        print("No sharps objects detected.")

In [5]:
# --- MAIN EXECUTION ---

# 1. Train the model (uncomment if you need to retrain)
# model = train_model(dataset_path)
# model.save(os.path.join(dataset_path, 'best.pt')) # Save the trained model

# 2. Load the trained model
model = YOLO(os.path.join(dataset_path, 'best.pt'))  # Load the trained model. Make sure the 'best.pt' file is in the correct directory.

# 3. Detect objects in the video
object_detected = detect_objects(model, video_path)

# print(object_detected)
# 4. Send email notification
# send_email(object_detected)

print("Finished.")

100%|██████████| 49.7M/49.7M [00:01<00:00, 33.6MB/s]


Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Colab Notebooks/fiap-hackathon-visionguard-content/dataset3/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

100%|██████████| 755k/755k [00:00<00:00, 120MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 379MB/s]


...

                   all        468        485        0.5      0.509      0.486      0.227

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      7.07G      1.481      1.422      1.823         11        640: 100%|██████████| 369/369 [01:52<00:00,  3.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.47it/s]


                   all        468        485      0.588      0.544       0.55      0.269

10 epochs completed in 0.332 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.39it/s]


                   all        468        485      0.586      0.544       0.55      0.269
Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train

0: 384x640 (no detections), 78.4ms
Speed: 1.9ms preprocess, 78.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.5ms
Speed: 2.0ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 1.9ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.4ms
Speed: 1.9ms preprocess, 9.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 1.9ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.4ms
Speed: 2.0ms preprocess, 9.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 38